Implementations of neural network layers based on computation graphs

In [2]:
import numpy as np

In [3]:
# Implements a multiplication layer
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self,x,y):
        self.x = x
        self.y = y
        out = x * y

        return out
    
    def backward(self, dout):
        # In backpropagation, multiply by the other input from the forward pass
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [4]:
# Example usage of MulLayer
apple = 100
apple_num = 2
tax = 1.1

# Forward
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple,apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

# Backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [5]:
# Implements a addition layer
class AddLayer:
    def __init__(self):
        pass

    def forward(self,x,y):
        return x+y
    
    def backward(self,dout):
        dx = dout*1
        dy = dout*1

        return dx, dy

In [6]:
# Implements a Relu layer
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self,x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx
    


In [7]:
# Implements a Sigmoid layer
class sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self,x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out)*self.out

        return dx
    

In [9]:
# Implements a Affine layer
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out
    
    def backward(self,dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

        

In [13]:
# Implements a Softmax-with-Loss layer
from functions import softmax, cross_entropy_error
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None 
        self.y = None # output of softmax
        self.t = None # true label

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
    

Example) An implementation of training algorithm using backpropagation

In [15]:
from gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # creates the layers
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x
    
    def loss(self, x, t):
        y = self.predcit(x)
        return self.last_layer.forward(y,t)
    
    def accuracy(self, x, t):
        y = self.predcit(x)
        y = np.argmax(y, axis=1)
        if t.ndim !=1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y==t) / float(x.shape[0])

    def gradient(self, x, t):
        self.loss(x,t)

        dout= 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].b1
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].b2

        return grads